In [528]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns

In [529]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score,recall_score,roc_auc_score,mean_absolute_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

In [530]:
train_data = pd.read_csv('./Desktop/train_data/train.csv')
customer_data = pd.read_csv('./Desktop/train_data/customer_demographics.csv')
transaction_data = pd.read_csv('./Desktop/train_data/customer_transaction_data.csv')
coupon_item_data = pd.read_csv('./Desktop/train_data/coupon_item_mapping.csv')
campaign_data = pd.read_csv('./Desktop/train_data/campaign_data.csv')
item_data = pd.read_csv('./Desktop/train_data/item_data.csv')

In [531]:
train_data.head()

,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0
2,6,9,635,205,0
3,7,13,644,1050,0
4,9,8,1017,1489,0


In [532]:
customer_data.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,NaN,4
1,6,46-55,Married,0,2,NaN,5
2,7,26-35,NaN,0,3,1,3
3,8,26-35,NaN,0,4,2,6
4,10,46-55,Single,0,1,NaN,5


In [533]:
train_customer_merged = pd.merge(train_data,customer_data, on=['customer_id'])

In [534]:
train_data['customer_id'].nunique()

1428

In [535]:
print(train_data.shape)
print(customer_data.shape)
print(train_customer_merged.shape)


(78369, 5)
(760, 7)
(43661, 11)


In [536]:
customer_data['customer_id'].nunique()

760

In [537]:
train_customer_merged['customer_id'].nunique()

703

In [538]:
train_data_d =  train_customer_merged.drop_duplicates(subset=['customer_id'])

In [539]:
train_data_d.shape

(703, 11)

In [540]:
a_index = td.set_index(['item_id']).index
b_index = coupon_item_data.set_index(['item_id']).index
mask = a_index.isin(b_index)
test_common = td.loc[mask]
print(test_common.shape)

(36286, 7)


In [541]:
a_index = transaction_data.set_index(['item_id']).index
b_index = cd.set_index(['item_id']).index
mask = b_index.isin(a_index)
test_common = cd.loc[mask]
print(test_common.shape)

(36286, 2)


In [542]:
a_index = transaction_data.set_index(['item_id']).index
b_index = coupon_item_data.set_index(['item_id']).index
mask = b_index.isin(a_index)
test_common = coupon_item_data.loc[mask]
print(test_common.shape)

(92655, 2)


In [543]:
test = pd.read_csv('./Desktop/test_data.csv')

In [544]:
trained_customer_data = train_customer_merged.drop(['marital_status','no_of_children'],axis=1)
trained_customer_data.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,age_range,rented,family_size,income_bracket
0,1,13,27,1053,0,46-55,0,1,5
1,2444,13,513,1053,0,46-55,0,1,5
2,3651,13,166,1053,0,46-55,0,1,5
3,7712,13,766,1053,0,46-55,0,1,5
4,15317,13,165,1053,0,46-55,0,1,5


In [545]:
coupon_item_data.nunique()

coupon_id     1116
item_id      36289
dtype: int64

In [546]:
transaction_data.head()


,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


In [547]:
trans_item_merge = pd.merge(transaction_data, coupon_item_data,on=['item_id'])

In [548]:
print(transaction_data.shape)
print(coupon_item_data.shape)
print(trans_item_merge.shape)


(1324566, 7)
(92663, 2)
(2101361, 8)


In [549]:
trans_item_merge1 = pd.merge(td, cd,on=['item_id'])

In [550]:
missing_cols = [col for col in trans_item_merge.columns if trans_item_merge[col].isnull().any() ]
missing_cols

[]

In [551]:
trans_item_merge.dropna()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,coupon_id
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0,7
1,2012-01-02,1501,26830,1,35.26,-10.69,0.0,20
2,2012-01-02,1501,26830,1,35.26,-10.69,0.0,29
3,2012-01-02,135,26830,1,35.26,-10.69,0.0,7
4,2012-01-02,135,26830,1,35.26,-10.69,0.0,20
...,...,...,...,...,...,...,...,...
2101356,2013-06-30,1129,2423,1,81.57,-7.12,0.0,211
2101357,2013-06-30,1129,2423,1,81.57,-7.12,0.0,30
2101358,2013-06-30,1129,2777,1,284.60,-71.24,0.0,43
2101359,2013-06-30,1129,2777,1,284.60,-71.24,0.0,42


In [552]:
print(transaction_data['item_id'].nunique())
print(coupon_item_data['item_id'].nunique())
print(trans_item_merge['item_id'].nunique())

74063
36289
36286


In [553]:
merged_on_customerIdAndCouponId = pd.merge(trans_item_merge, trained_customer_data, on=['customer_id', 'coupon_id'])

In [554]:
merged_on_customerIdAndCouponId.drop(['id','date'], axis=1,inplace=True)

In [555]:
merged_on_customerIdAndCouponId.head()

,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,coupon_id,campaign_id,redemption_status,age_range,rented,family_size,income_bracket
0,11,26830,1,53.07,0.00,0.0,7,8,0,70+,0,2,1
1,11,26830,1,53.07,0.00,0.0,7,8,0,70+,0,2,1
2,11,13174,1,35.26,0.00,0.0,7,8,0,70+,0,2,1
3,11,13174,1,35.26,0.00,0.0,7,8,0,70+,0,2,1
4,11,13174,1,24.58,-10.69,0.0,7,8,0,70+,0,2,1


In [556]:
campaign_data['campaign_type'] = campaign_data['campaign_type'].replace({'X':0,'Y':1})

In [557]:
campaign_data_without_dates = campaign_data.drop(['start_date', 'end_date'], axis=1)

In [558]:
campaign_data_without_dates.head()

,campaign_id,campaign_type
0,24,1
1,25,1
2,20,1
3,23,1
4,21,1


In [559]:
merged_on_item = pd.merge(merged_on_customerIdAndCouponId,item_data, on=['item_id'])


In [560]:
merged_on_item.head()

,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,coupon_id,campaign_id,redemption_status,age_range,rented,family_size,income_bracket,brand,brand_type,category
0,11,26830,1,53.07,0.00,0.0,7,8,0,70+,0,2,1,56,Local,Natural Products
1,11,26830,1,53.07,0.00,0.0,7,8,0,70+,0,2,1,56,Local,Natural Products
2,1473,26830,1,53.07,0.00,0.0,7,8,0,36-45,0,1,5,56,Local,Natural Products
3,103,26830,1,17.45,-35.62,0.0,7,8,0,46-55,0,2,4,56,Local,Natural Products
4,103,26830,1,45.95,0.00,0.0,7,8,0,46-55,0,2,4,56,Local,Natural Products


In [561]:
merged_on_campaign = pd.merge(merged_on_item,campaign_data_without_dates, on='campaign_id')

In [562]:
merged_on_campaign.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94855 entries, 0 to 94854
Data columns (total 17 columns):
customer_id          94855 non-null int64
item_id              94855 non-null int64
quantity             94855 non-null int64
selling_price        94855 non-null float64
other_discount       94855 non-null float64
coupon_discount      94855 non-null float64
coupon_id            94855 non-null int64
campaign_id          94855 non-null int64
redemption_status    94855 non-null int64
age_range            94855 non-null object
rented               94855 non-null int64
family_size          94855 non-null object
income_bracket       94855 non-null int64
brand                94855 non-null int64
brand_type           94855 non-null object
category             94855 non-null object
campaign_type        94855 non-null int64
dtypes: float64(3), int64(10), object(4)
memory usage: 13.0+ MB


In [563]:
merged = merged_on_campaign.copy()

In [564]:
merged.drop(['item_id'],axis=1,inplace=True)

In [565]:
coupon_item_data.nunique()

coupon_id     1116
item_id      36289
dtype: int64

In [566]:
test = pd.read_csv('./Desktop/test_data.csv')

In [567]:
test.head()

,id,campaign_id,coupon_id,customer_id
0,3,22,869,967
1,4,20,389,1566
2,5,22,981,510
3,8,25,1069,361
4,10,17,498,811


In [568]:
test.shape

(50226, 4)

In [569]:
merged.head()

,customer_id,quantity,selling_price,other_discount,coupon_discount,coupon_id,campaign_id,redemption_status,age_range,rented,family_size,income_bracket,brand,brand_type,category,campaign_type
0,11,1,53.07,0.00,0.0,7,8,0,70+,0,2,1,56,Local,Natural Products,0
1,11,1,53.07,0.00,0.0,7,8,0,70+,0,2,1,56,Local,Natural Products,0
2,1473,1,53.07,0.00,0.0,7,8,0,36-45,0,1,5,56,Local,Natural Products,0
3,103,1,17.45,-35.62,0.0,7,8,0,46-55,0,2,4,56,Local,Natural Products,0
4,103,1,45.95,0.00,0.0,7,8,0,46-55,0,2,4,56,Local,Natural Products,0


In [570]:
training = merged.copy()

In [571]:
X = merged.copy()
train_with_all_cols = training.copy()
X.drop(['redemption_status'],axis=1,inplace=True)

X_test = test.copy()
y= training['redemption_status']
# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

In [572]:
X_train_with_ids = X_train[['customer_id','coupon_id','campaign_id']]
X_train = X_train.drop(['customer_id','coupon_id','campaign_id'],axis=1)

X_valid_with_ids = X_valid[['customer_id','coupon_id','campaign_id']]
X_valid = X_valid.drop(['customer_id','coupon_id','campaign_id'],axis=1)


In [573]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75884 entries, 60755 to 68268
Data columns (total 12 columns):
quantity           75884 non-null int64
selling_price      75884 non-null float64
other_discount     75884 non-null float64
coupon_discount    75884 non-null float64
age_range          75884 non-null object
rented             75884 non-null int64
family_size        75884 non-null object
income_bracket     75884 non-null int64
brand              75884 non-null int64
brand_type         75884 non-null object
category           75884 non-null object
campaign_type      75884 non-null int64
dtypes: float64(3), int64(5), object(4)
memory usage: 7.5+ MB


In [574]:
s = (X_train.dtypes == 'object')
object_cols = [s[s].index]

print("Categorical variables:")
print(object_cols)

Categorical variables:
[Index(['age_range', 'family_size', 'brand_type', 'category'], dtype='object')]


In [575]:
object_cols = ['age_range', 'family_size', 'brand_type', 'category']

In [576]:
#OneHotEncoding
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

In [577]:
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Columns that can be safely label encoded
good_label_cols = [col for col in object_cols if 
                   set(X_train[col]) == set(X_valid[col])]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = [set(object_cols)-set(good_label_cols)]
#print(X_train['Norm'].unique())   
print('Categorical columns that will be label encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

Categorical columns that will be label encoded: ['age_range', 'family_size', 'brand_type']

Categorical columns that will be dropped from the dataset: [{'category'}]


In [578]:
bad_label_cols=['category']

In [579]:
#LabelEncoding
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

# Apply label encoder 
label_encoder = LabelEncoder()
for col in good_label_cols:
    label_X_train[col] = label_encoder.fit_transform(label_X_train[col])
    label_X_valid[col] = label_encoder.transform(label_X_valid[col])
    print(label_X_valid[col].unique())

[0 3 2 1 5 4]
[0 1 2 3 4]
[0 1]


In [580]:
print(X_train.columns)
print(OH_X_train.columns)

Index(['quantity', 'selling_price', 'other_discount', 'coupon_discount',
       'age_range', 'rented', 'family_size', 'income_bracket', 'brand',
       'brand_type', 'category', 'campaign_type'],
      dtype='object')
Index([       'quantity',   'selling_price',  'other_discount',
       'coupon_discount',          'rented',  'income_bracket',
                 'brand',   'campaign_type',                 0,
                       1,                 2,                 3,
                       4,                 5,                 6,
                       7,                 8,                 9,
                      10,                11,                12,
                      13,                14,                15,
                      16,                17,                18,
                      19,                20,                21,
                      22,                23,                24,
                      25,                26,                27,
              

In [581]:
OH_X_train.describe()

,quantity,selling_price,other_discount,coupon_discount,rented,income_bracket,brand,campaign_type,0,1,...,19,20,21,22,23,24,25,26,27,28
count,75884.000000,75884.000000,75884.000000,75884.000000,75884.000000,75884.000000,75884.000000,75884.000000,75884.00000,75884.000000,...,75884.000000,75884.000000,75884.000000,75884.000000,75884.000000,75884.000000,75884.000000,75884.000000,75884.000000,75884.000000
mean,1.331163,108.187544,-22.121508,-1.435141,0.055242,5.014180,884.594592,0.198487,0.04889,0.206855,...,0.000514,0.063861,0.086751,0.132439,0.001621,0.000369,0.010964,0.003716,0.000343,0.000053
std,0.905974,105.077905,38.796969,8.373478,0.228454,2.314387,1047.375631,0.398864,0.21564,0.405053,...,0.022665,0.244506,0.281471,0.338970,0.040228,0.019206,0.104135,0.060848,0.018507,0.007260
min,1.000000,1.420000,-1175.100000,-534.300000,0.000000,1.000000,1.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,53.430000,-32.060000,0.000000,0.000000,4.000000,89.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,88.690000,-7.120000,0.000000,0.000000,5.000000,681.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,124.670000,0.000000,0.000000,0.000000,6.000000,1149.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,54.000000,3846.960000,0.000000,0.000000,1.000000,12.000000,5468.000000,1.000000,1.00000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [582]:
cols_to_scale = ['quantity','selling_price','coupon_discount','income_bracket','brand']
OH_X_train_left= OH_X_train.drop(cols_to_scale, axis=1)
OH_X_valid_left= OH_X_valid.drop(cols_to_scale, axis=1)

OH_X_train_scale = OH_X_train[cols_to_scale]
OH_X_valid_scale = OH_X_valid[cols_to_scale]

OH_X_valid_scale.shape

(18971, 5)

In [583]:
label_X_train_l = label_X_train.drop(cols_to_scale, axis=1)
label_X_valid_l = label_X_valid.drop(cols_to_scale, axis=1)

label_X_train_scale = label_X_train[cols_to_scale]
label_X_valid_scale = label_X_valid[cols_to_scale]

label_X_train_l.shape

(75884, 6)

In [584]:
print(type(cols_to_scale))
OH_X_train_scaled = StandardScaler().fit_transform(OH_X_train_scale)
OH_X_valid_scaled = StandardScaler().fit_transform(OH_X_valid_scale)

train_scaled = pd.DataFrame(OH_X_train_scaled, columns=['quantity','selling_price','coupon_discount','income_bracket','brand'])
train_scaled_v = pd.DataFrame(OH_X_valid_scaled, columns=['quantity','selling_price','coupon_discount','income_bracket','brand'])

OH_X_train_s = pd.DataFrame(np.hstack([OH_X_train_scaled,OH_X_train_left]))
OH_X_valid_s = pd.DataFrame(np.hstack([OH_X_valid_scaled,OH_X_valid_left]))

#OH_X_train_s.columns = 
#OH_X_valid_s.columns = [cols_to_scale + OH_X_valid_left.columns]

#OH_X_train_s = pd.concat([train_scaled, OH_X_train_left],axis=1)

OH_X_valid_s.shape

<class 'list'>


(18971, 37)

In [585]:
label_X_train_scaled = StandardScaler().fit_transform(label_X_train_scale)
label_X_valid_scaled = StandardScaler().fit_transform(label_X_valid_scale)

label_X_train_s = pd.DataFrame(np.hstack([label_X_train_scaled,label_X_train_l]))#,columns = cols_to_scale)
label_X_valid_s = pd.DataFrame(np.hstack([label_X_valid_scaled,label_X_valid_l]))#,columns = cols_to_scale)

label_X_train_s.shape

(75884, 11)

In [586]:
OH_X_train_s.dropna()
OH_X_train_s.head()

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,-0.365536,3.373828,0.171392,-1.734458,-0.791120,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.365536,-0.351623,0.171392,-0.006127,-0.162879,-13.89,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.365536,-0.694038,0.171392,-0.438209,0.375613,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.365536,-0.117700,0.171392,-0.438209,-0.615442,-10.69,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.365536,-0.219434,0.171392,-0.006127,-0.791120,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [587]:
clf = RandomForestClassifier()

In [588]:
tree = DecisionTreeClassifier()

In [589]:
svm = SVC()

In [590]:
def score_model_for(model,X_train,X_valid,y_train,y_valid):
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [591]:
print(score_model_for(clf,label_X_train,label_X_valid,y_train,y_valid))
print(score_model_for(tree,label_X_train,label_X_valid,y_train,y_valid))
#print(score_model_for(svm,label_X_train,label_X_valid,y_train,y_valid))
print(score_model_for(clf,label_X_train_s,label_X_valid_s,y_train,y_valid))
print(score_model_for(tree,label_X_train_s,label_X_valid_s,y_train,y_valid))


C:\Users\vh6tgd\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.06757682778978441
0.07137209424911707
0.07875177903115281
0.09092825892151178


In [592]:
print(confusion_matrix(y_valid,clf.predict(label_X_valid)))
print(confusion_matrix(y_valid,tree.predict(label_X_valid)))

print(confusion_matrix(y_valid,clf.predict(label_X_valid_s)))
print(confusion_matrix(y_valid,tree.predict(label_X_valid_s)))

#print(confusion_matrix(y_valid,svm.predict(label_X_valid)))

[[15644     0]
 [ 3327     0]]
[[11477  4167]
 [ 2491   836]]
[[15176   468]
 [ 1026  2301]]
[[14920   724]
 [ 1001  2326]]


In [593]:
print(roc_auc_score(y_valid,clf.predict(label_X_valid)))
print(roc_auc_score(y_valid,tree.predict(label_X_valid)))

print(roc_auc_score(y_valid,clf.predict(label_X_valid_s)))
print(roc_auc_score(y_valid,tree.predict(label_X_valid_s)))
#print(roc_auc_score(y_valid,svm.predict(label_X_valid)))

0.5
0.49245666292931767
0.8308492220619331
0.8264243099987649


In [594]:
print(score_model_for(clf,OH_X_train,OH_X_valid,y_train,y_valid))
print(score_model_for(tree,OH_X_train,OH_X_valid,y_train,y_valid))

print(score_model_for(clf,OH_X_train_s,OH_X_valid_s,y_train,y_valid))
print(score_model_for(tree,OH_X_train_s,OH_X_valid_s,y_train,y_valid))
#score_model_for(svm,OH_X_train,OH_X_valid,y_train,y_valid)

0.06130409572505403
0.06625902693584945
0.06631173897000685
0.08064941226081915


In [595]:
print(confusion_matrix(y_valid,clf.predict(OH_X_valid)))
print(confusion_matrix(y_valid,tree.predict(OH_X_valid)))

print(confusion_matrix(y_valid,clf.predict(OH_X_valid_s)))
print(confusion_matrix(y_valid,tree.predict(OH_X_valid_s)))
#print(confusion_matrix(y_valid,svm.predict(OH_X_valid)))

[[11199  4445]
 [ 1732  1595]]
[[11704  3940]
 [ 2249  1078]]
[[15198   446]
 [  812  2515]]
[[14999   645]
 [  885  2442]]


In [596]:
print(roc_auc_score(y_valid,clf.predict(OH_X_valid)))
print(roc_auc_score(y_valid,tree.predict(OH_X_valid)))

print(roc_auc_score(y_valid,clf.predict(OH_X_valid_s)))
print(roc_auc_score(y_valid,tree.predict(OH_X_valid_s)))
#print(roc_auc_score(y_valid,svm.predict(OH_X_valid)))

0.5976381941080536
0.5360809419256853
0.8637134731392355
0.8463823626178412


In [597]:
from sklearn.model_selection import GridSearchCV

In [598]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

In [599]:
param_grid = {'max_features': ['auto', 'sqrt', 'log2']}

grid_search = GridSearchCV(clf, param_grid)
grid_search.fit(OH_X_train_s,y_train)

# Print the optimal parameters
print(grid_search.best_params_)

C:\Users\vh6tgd\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


{'max_features': 'log2'}


In [600]:
param_dist = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

random_search = RandomizedSearchCV(clf,param_dist)

random_search.fit(OH_X_train_s,y_train)
#random_search.fit(label_X_train_s,y_train)
print(random_search.best_params_)

C:\Users\vh6tgd\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


{'max_features': 10, 'max_depth': None, 'criterion': 'gini', 'bootstrap': False}


In [815]:
#clf = RandomForestClassifier(max_features= 10,max_depth=None,criterion= 'gini',bootstrap=False)
#clf.fit(OH_X_train,y_train)

TypeError: __init__() got an unexpected keyword argument 'bootstrap'

In [817]:
clf = DecisionTreeClassifier()
clf.fit(OH_X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [818]:
print(OH_X_train_s.shape)
print(X_valid_with_ids.shape)
type(X_train_with_ids.columns)
list1 = [X_train_with_ids.columns]#.add(OH_X_train_s.columns)
list2 = [OH_X_train_s.columns]
all_columns = list1 + list2

#X_train_all_cols = pd.concat([X_train_with_ids, OH_X_train_s], axis=1)
X_train_all_cols = X_train_with_ids.join(OH_X_train)
X_valid_all_cols = X_valid_with_ids.join(OH_X_valid)

#X_train_all_cols = pd.DataFrame(np.column_stack([X_train_with_ids,OH_X_train_s]),columns = all_columns)
#X_valid_all_cols = pd.DataFrame(np.hstack([X_valid_with_ids,OH_X_valid_s]),columns = all_columns)
X_valid_all_cols.shape
#type(np.hstack([X_train_with_ids,OH_X_train_s]))

(75884, 37)
(18971, 3)


(18971, 40)

In [819]:
OH_X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75884 entries, 60755 to 68268
Data columns (total 37 columns):
quantity           75884 non-null int64
selling_price      75884 non-null float64
other_discount     75884 non-null float64
coupon_discount    75884 non-null float64
rented             75884 non-null int64
income_bracket     75884 non-null int64
brand              75884 non-null int64
campaign_type      75884 non-null int64
0                  75884 non-null float64
1                  75884 non-null float64
2                  75884 non-null float64
3                  75884 non-null float64
4                  75884 non-null float64
5                  75884 non-null float64
6                  75884 non-null float64
7                  75884 non-null float64
8                  75884 non-null float64
9                  75884 non-null float64
10                 75884 non-null float64
11                 75884 non-null float64
12                 75884 non-null float64
13             

In [820]:

#X_with_encodings = pd.DataFrame(np.hstack([X_train_all_cols,X_valid_all_cols]))
X_with_encodings = X_train_all_cols.append(X_valid_all_cols)
#rained_set = pd.DataFrame(X_with_encodings, columns=[X_train_all_cols.columns])
print(X_with_encodings.shape)
trained_set = X_with_encodings

(94855, 40)


In [821]:
X_customers_coupon_campaign = trained_set[['customer_id','coupon_id','campaign_id']] 
X_customers = trained_set[['customer_id']]
X_customers_coupon = trained_set[['customer_id','coupon_id']]
X_customers_campaign = trained_set[['customer_id','campaign_id']]
X_campaign_coupon = trained_set[['coupon_id','campaign_id']]

In [822]:
merged_with_test_customers = pd.merge(trained_set,test,on=['customer_id']) 

In [823]:
merged_with_test_customers.shape

(5387543, 43)

In [824]:
test_customers = test[['customer_id']]

In [825]:
test_customer_coupons_in_X = pd.merge(test, X_customers_coupon, on= ['customer_id','coupon_id'])

In [826]:
test_data_in_X_encodings = pd.merge(test, X_with_encodings, on= ['customer_id','coupon_id','campaign_id'])#,how='left')

In [827]:
test_data_in_X_encodings.shape

(0, 41)

In [828]:
test_customer_coupons_in_X_encodings = pd.merge(test, X_with_encodings, on= ['customer_id','coupon_id'])
#test_customer_coupons_in_X_encodings_1 = pd.concat([test, X_with_encodings], on= ['customer_id','coupon_id'],ignore_index=True)#,how='left'),ignore_index=True)#,how='left')

In [829]:
print(test_customer_coupons_in_X_encodings.shape)
test_customer_coupons_in_X_encodings.head()

(4942, 42)


,id,campaign_id_x,coupon_id,customer_id,campaign_id_y,quantity,selling_price,other_discount,coupon_discount,rented,...,19,20,21,22,23,24,25,26,27,28
0,114,19,689,28,5,1,569.56,0.0,0.0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,114,19,689,28,5,1,377.22,0.0,0.0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,114,19,689,28,5,1,533.94,0.0,0.0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,114,19,689,28,5,1,533.94,0.0,0.0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,114,19,689,28,5,1,533.94,0.0,0.0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [830]:
x = test_customer_coupons_in_X_encodings.drop_duplicates(subset = ['customer_id','coupon_id'])
x.shape

(372, 42)

In [831]:
y = X_with_encodings.drop_duplicates(subset = ['customer_id','coupon_id'])
y.shape

(8199, 40)

In [832]:
a_index = X_with_encodings.set_index(['customer_id','coupon_id']).index
b_index = test.set_index(['customer_id','coupon_id']).index
mask = a_index.isin(b_index)
mask_2 = b_index.isin(a_index)
test_customer_coupons_in_X_encodings1 = X_with_encodings.loc[mask]
ids = test.loc[mask_2]['id']
print(ids.shape)
test_ids = pd.DataFrame(ids, columns =['id'])

#test_customer_coupons_in_X_encodings = pd.concat([test_customer_coupons_in_X_encodings, test_ids], axis=1)

print(test_customer_coupons_in_X_encodings1.shape)
print(test_customer_coupons_in_X_encodings1.head())

(386,)
(4869, 40)
       customer_id  coupon_id  campaign_id  quantity  selling_price  \
71256         1070        754           13         1         102.59   
91216          389        934           10         1          14.96   
92555          626        934            5         2          28.50   
92393          377        934            5         1          14.96   
70633         1387        942           13         1          89.05   

       other_discount  coupon_discount  rented  income_bracket  brand  ...  \
71256          -89.41           -35.62       0               6    864  ...   
91216            0.00             0.00       0               6   1149  ...   
92555            0.00             0.00       0               4   1149  ...   
92393            0.00             0.00       1               4   1149  ...   
70633          -45.95           -35.62       0               1   1183  ...   

        19   20   21   22   23   24   25   26   27   28  
71256  0.0  0.0  0.0  0.0  0

In [833]:
test_customer_coupons_in_X_encodings.shape

(4942, 42)

In [834]:
partial = test_customer_coupons_in_X_encodings[['coupon_id', 'customer_id']]
partial.shape

(4942, 2)

In [835]:
common_customer_coupons = test_customer_coupons_in_X_encodings.rename(columns={'campaign_id_x':'campaign_id'})
common_set = common_customer_coupons.drop('campaign_id_y',axis=1)
#ommon_customer_coupons_imp = common_customer_coupons[[common_customer_coupons['campaign_id']==common_customer_coupons['campaign_id_y']]]
print(common_customer_coupons['campaign_id'].isna().sum())
print(common_set.shape)
common_set.info()

0
(4942, 41)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4942 entries, 0 to 4941
Data columns (total 41 columns):
id                 4942 non-null int64
campaign_id        4942 non-null int64
coupon_id          4942 non-null int64
customer_id        4942 non-null int64
quantity           4942 non-null int64
selling_price      4942 non-null float64
other_discount     4942 non-null float64
coupon_discount    4942 non-null float64
rented             4942 non-null int64
income_bracket     4942 non-null int64
brand              4942 non-null int64
campaign_type      4942 non-null int64
0                  4942 non-null float64
1                  4942 non-null float64
2                  4942 non-null float64
3                  4942 non-null float64
4                  4942 non-null float64
5                  4942 non-null float64
6                  4942 non-null float64
7                  4942 non-null float64
8                  4942 non-null float64
9                  4942 non-null floa

In [836]:
from sklearn.impute import SimpleImputer

In [837]:
cols_with_nan = [col for col in test_customer_coupons_in_X_encodings.columns if test_customer_coupons_in_X_encodings[col].isnull().any()]
len(cols_with_nan)

0

In [838]:
data_to_check = common_set.copy()

In [839]:
#test_customer_coupons_in_X_encodings.rename(columns={'campaign_id_x':'campaign_id'})
test_for_pred = data_to_check.drop(['campaign_id','coupon_id','customer_id','id'],axis=1)
test_with_ids = data_to_check[['coupon_id','customer_id','campaign_id','id']] 
data_to_check.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4942 entries, 0 to 4941
Data columns (total 41 columns):
id                 4942 non-null int64
campaign_id        4942 non-null int64
coupon_id          4942 non-null int64
customer_id        4942 non-null int64
quantity           4942 non-null int64
selling_price      4942 non-null float64
other_discount     4942 non-null float64
coupon_discount    4942 non-null float64
rented             4942 non-null int64
income_bracket     4942 non-null int64
brand              4942 non-null int64
campaign_type      4942 non-null int64
0                  4942 non-null float64
1                  4942 non-null float64
2                  4942 non-null float64
3                  4942 non-null float64
4                  4942 non-null float64
5                  4942 non-null float64
6                  4942 non-null float64
7                  4942 non-null float64
8                  4942 non-null float64
9                  4942 non-null float64
10       

In [840]:

def impute(test_for_pred):    
    my_imputer = SimpleImputer()
    imputed_X_train = pd.DataFrame(my_imputer.fit_transform(test_for_pred))
    #imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

    # Imputation removed column names; put them back
    imputed_X_train.columns = test_for_pred.columns
    return imputed_X_train

In [841]:
#imputed_X_train = impute(test_for_pred)

In [842]:
def numOfNanIn(train):
    cols_with_nan = [col for col in train.columns if train[col].isnull().any()]
    return len(cols_with_nan)

In [843]:
print(numOfNanIn(imputed_X_train))

0


In [844]:
pred = clf.predict(imputed_X_train)

In [845]:
def createDFOf(pred):
    preds = pd.DataFrame(pred, columns=['redemption_status'])
    return preds

In [846]:
preds = createDFOf(pred)
preds.shape

(4942, 1)

In [847]:
def partOfResult(test_with_ids,preds):
    part_of_result = pd.concat([test_with_ids,preds],axis=1)
    return part_of_result

In [848]:
part_of_result = partOfResult(test_with_ids,preds)
part_of_result.shape

(4942, 5)

In [849]:
res = pd.merge(test,part_of_result,on=['customer_id','coupon_id'])
res.shape

(5088, 7)

In [850]:
res.columns

Index(['id_x', 'campaign_id_x', 'coupon_id', 'customer_id', 'campaign_id_y',
       'id_y', 'redemption_status'],
      dtype='object')

In [851]:
subset = res[res.campaign_id_x == res.campaign_id_y]
subset.shape

(4964, 7)

In [852]:
result_1 = subset[['id_x','redemption_status']]

In [853]:
result0 = result_1.rename(columns={'id_x':'id'})

In [854]:
result0.shape

(4964, 2)

In [855]:
result0['redemption_status'].value_counts()

0    4035
1     929
Name: redemption_status, dtype: int64

In [856]:
result0.to_csv('partial_submission.csv')

In [857]:

sub = subset.rename(columns={'id_x':'id','campaign_id_x':'campaign_id'})

In [858]:
index_a = test.set_index(['customer_id','coupon_id','campaign_id','id']).index
index_b = subset.set_index(['customer_id','coupon_id','campaign_id_y','id_y']).index
#index_c = X_with_encodings.set_index(['customer_id','coupon_id','campaign_id','id']).index
mask = ~index_a.isin(index_b)

#mask2= ~index_a.isin(index_c)
#mask3= ~index_c.isin(index_a)

partial_test = test.loc[mask]
#partial_train = X_with_encodings.loc[mask3]
partial_test.shape

(49840, 4)

In [859]:
partial_test.columns

Index(['id', 'campaign_id', 'coupon_id', 'customer_id'], dtype='object')

In [860]:
reducedX = X_with_encodings.drop_duplicates(subset=['customer_id','campaign_id'])
reducedX.shape

(2160, 40)

In [861]:
m1 = pd.merge(partial_test,X_customers_campaign,on=['customer_id'])
m1.shape

(5283067, 5)

In [862]:
m1 = pd.merge(partial_test,reducedX,on=['customer_id'])
m1.shape

(101716, 43)

In [863]:
index_a = m1.set_index(['customer_id']).index
index_b = subset.set_index(['customer_id']).index
#index_c = X_with_encodings.set_index(['customer_id','coupon_id','campaign_id','id']).index
mask = ~index_a.isin(index_b)

#mask2= ~index_a.isin(index_c)
#mask3= ~index_c.isin(index_a)

partial_test1 = m1.loc[mask]
#partial_train = X_with_encodings.loc[mask3]
partial_test1.shape

(46784, 43)

In [864]:
m1 = partial_test1.copy()

In [865]:
print(numOfNanIn(m1))

0


In [866]:
m1.columns

Index([             'id',   'campaign_id_x',     'coupon_id_x',
           'customer_id',     'coupon_id_y',   'campaign_id_y',
              'quantity',   'selling_price',  'other_discount',
       'coupon_discount',          'rented',  'income_bracket',
                 'brand',   'campaign_type',                 0,
                       1,                 2,                 3,
                       4,                 5,                 6,
                       7,                 8,                 9,
                      10,                11,                12,
                      13,                14,                15,
                      16,                17,                18,
                      19,                20,                21,
                      22,                23,                24,
                      25,                26,                27,
                      28],
      dtype='object')

In [867]:
test_pred = m1.drop(['campaign_id_x','coupon_id_x','customer_id','id','coupon_id_y','campaign_id_y'],axis=1)
test_with_ids = m1[['coupon_id_x','customer_id','campaign_id_x','id']] 

In [868]:
test_with_ids.head()

,coupon_id_x,customer_id,campaign_id_x,id
1132,1069,361,25,8
1133,1069,361,25,8
1134,596,361,25,679
1135,596,361,25,679
1136,612,361,25,5354


In [869]:
imputed = impute(test_pred)

In [870]:
prediction = clf.predict(imputed)

In [871]:
 pred_df = createDFOf(prediction)

In [872]:
pred_df.head()

,redemption_status
0,0
1,0
2,0
3,0
4,0


In [873]:
#res = pd.merge(test_with_ids,pred_df,axis=1)
res = test_with_ids.join(pred_df)
#res = partOfResult(test_with_ids,pred_df)
res.shape

(46784, 5)

In [874]:
index_a = res.set_index(['id']).index
index_b = test.set_index(['id']).index
#index_c = X_with_encodings.set_index(['customer_id','coupon_id','campaign_id','id']).index
mask = index_a.isin(index_b)

#mask2= ~index_a.isin(index_c)
#mask3= ~index_c.isin(index_a)

partial_test_1 = res.loc[mask]
#partial_train = X_with_encodings.loc[mask3]
partial_test_1.shape

(46784, 5)

In [875]:
final_final = res[['id','redemption_status']]
final_final.shape

(46784, 2)

In [876]:
combined = pd.concat([result0,final_final])

In [877]:
combined.shape

(51748, 2)

In [878]:
new = test.merge(combined,how='left',on=['id'], indicator=True) # adds a new column '_merge'
new = new[(new['_merge']=='left_only')].copy() #rows only in df1 and not df2
new = new.drop(columns='_merge').copy()
new.shape

(33714, 5)

In [879]:
c = combined.drop_duplicates(subset=['id'])

In [880]:
a_index = test.set_index(['id']).index
b_index = c.set_index(['id']).index
mask = a_index.isin(b_index)
partial_test_1 = test.loc[mask]
print(partial_test_1.shape)

(16512, 4)


In [881]:
df = pd.merge(partial_test_1,combined,on=['id'],how='left')
df.shape

(51748, 5)

In [882]:
dropped = df.drop_duplicates(subset = ['id'])
dropped.shape

(16512, 5)

In [883]:
submission = dropped[['id','redemption_status']]
submission.shape

(16512, 2)

In [884]:
res.columns

Index(['coupon_id_x', 'customer_id', 'campaign_id_x', 'id',
       'redemption_status'],
      dtype='object')

In [885]:
subset_s = sub[['id','customer_id','redemption_status']]
res_s = res[['id','customer_id','redemption_status']]

In [886]:
dropped1 = subset_s.drop_duplicates(subset = ['id'])
dropped2 = res_s.drop_duplicates(subset = ['id'])

In [887]:
checker = pd.concat([dropped1,dropped2])
checker.shape

(16512, 3)

In [888]:
a_index = test.set_index(['id']).index
b_index = checker.set_index(['id']).index
mask = ~a_index.isin(b_index)
remaining_in_test_final = test.loc[mask]
print(remaining_in_test_final.shape)

(33714, 4)


In [889]:
dict2 = zip(checker['customer_id'],checker['redemption_status'])
df = remaining_in_test_final.copy()

In [890]:
#df = remaining_in_test_final.copy()
df['redemption_status'] = np.zeros(len(df))
for key,val in dict2:
    df.loc[df['customer_id']==key, 'redemption_status'] = val 
    df.loc[df['customer_id']!=key, 'redemption_status'] = [0]

In [891]:
#for i in range(len(remaining_in_test_final)):

#df.loc[df['customer_id'].isin([checker['customer_id']]),'redemption_status'] = checker['redemption_status']
#df.loc[df['customer_id'].isin([checker['customer_id']]),'redemption_status'] = [0]

In [892]:
submit2 = df[['id','redemption_status']]

In [893]:
submission_final = pd.concat([submission,submit2])
submission_final.shape

(50226, 2)

In [894]:
print(numOfNanIn(submission_final))

1


In [895]:
submission_final['redemption_status'].fillna(value=0, inplace=True)

In [896]:
print(numOfNanIn(submission_final))

0


In [898]:
submission_final.to_csv('./Desktop/sub.csv')